In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# ==========================================
# 1. SIMULASI DATA DARI SUPABASE (MOCKUP)
# ==========================================

# Mock Table: SHIFT
# Weekday: 3 Shift @ 480 menit (8 jam)
# Saturday: 3 Shift @ 300 menit (5 jam)
shift_data = [
    {'shift_id': 1, 'shift_name': 'Shift 1', 'day_type': 'WEEKDAY',  'start_time': '07:00', 'end_time': '15:00', 'total_minutes': 420},
    {'shift_id': 2, 'shift_name': 'Shift 2', 'day_type': 'WEEKDAY',  'start_time': '15:00', 'end_time': '23:00', 'total_minutes': 420},
    {'shift_id': 3, 'shift_name': 'Shift 3', 'day_type': 'WEEKDAY',  'start_time': '23:00', 'end_time': '07:00', 'total_minutes': 420},
    {'shift_id': 4, 'shift_name': 'Shift 1', 'day_type': 'SATURDAY', 'start_time': '07:00', 'end_time': '12:00', 'total_minutes': 300},
    {'shift_id': 5, 'shift_name': 'Shift 2', 'day_type': 'SATURDAY', 'start_time': '12:00', 'end_time': '17:00', 'total_minutes': 300},
    {'shift_id': 6, 'shift_name': 'Shift 3', 'day_type': 'SATURDAY', 'start_time': '17:00', 'end_time': '22:00', 'total_minutes': 300},
]
df_shift = pd.DataFrame(shift_data)

# Mock View: v_ai_production_requirement
# Ini requirement produksi yang harus dijadwalkan
req_data = [
    # Part Avanza butuh 2000 menit (sekitar 4 shift lebih)
    {'part_no': '7437A976XA', 'part_name': 'Part 116 for Avanza', 'default_machine': '250T', 'estimated_minutes_required': 2000, 'shortage_flag': True},
    # Part Pajero butuh 1000 menit
    {'part_no': 'MSB55992',   'part_name': 'Part 63 for Pajero',  'default_machine': '250T', 'estimated_minutes_required': 1000, 'shortage_flag': False},
]
df_req = pd.DataFrame(req_data)

# ==========================================
# 2. LOGIC GENERATOR SLOT WAKTU (TIMELINE)
# ==========================================

def generate_production_timeline(start_date_str, days, df_shift_rules):
    start_date = pd.to_datetime(start_date_str)
    timeline_slots = []
    
    # Loop tanggal 1 sampai sekian
    for i in range(days):
        current_date = start_date + timedelta(days=i)
        day_name = current_date.strftime('%A') # Monday, Tuesday...
        
        # Tentukan Tipe Hari (Logic Supabase lo)
        current_day_type = 'WEEKDAY'
        if day_name == 'Saturday':
            current_day_type = 'SATURDAY'
        elif day_name == 'Sunday':
            current_day_type = 'SUNDAY' # Libur
            
        # Skip hari Minggu (Libur)
        if current_day_type == 'SUNDAY':
            continue
            
        # Ambil Rules Shift yang sesuai hari ini
        daily_rules = df_shift_rules[df_shift_rules['day_type'] == current_day_type]
        
        for _, rule in daily_rules.iterrows():
            # Bikin ID Unik buat setiap slot: Tgl_Shift
            slot_id = f"{current_date.strftime('%Y-%m-%d')}_{rule['shift_name']}"
            
            timeline_slots.append({
                'date': current_date,
                'day_name': day_name,
                'shift_name': rule['shift_name'],
                'duration_min': rule['total_minutes'], # Ini yang dinamis (480 vs 300)
                'slot_id': slot_id,
                'machine_id': '250T' # Nanti ini bisa diloop per mesin
            })
            
    return pd.DataFrame(timeline_slots)

# ==========================================
# 3. TEST RUN
# ==========================================
# Generate jadwal untuk bulan November 2025
df_timeline = generate_production_timeline('2025-11-01', 30, df_shift)

print("=== CONTOH TIMELINE YANG TERBENTUK ===")
# Cek apakah Sabtu beneran 300 menit?
print(df_timeline[df_timeline['day_name'].isin(['Friday', 'Saturday'])].head(10))

In [ ]:
from ortools.sat.python import cp_model

# ==========================================
# 4. OR-TOOLS SOLVER ENGINE
# ==========================================

def solve_production_schedule(df_slots, df_requirements):
    model = cp_model.CpModel()
    
    # --- A. VARIABLES (Kotak Kosong yang mau diisi AI) ---
    # format: x[part_index, slot_index] = berapa menit part ini jalan di shift itu?
    production_vars = {} 
    
    # Kita butuh index angka buat loop, bukan string
    parts_indices = df_requirements.index.tolist()
    slots_indices = df_slots.index.tolist()

    for p_idx in parts_indices:
        for s_idx in slots_indices:
            # Variable: Jumlah menit (0 sampai Kapasitas Slot tersebut)
            # Logika: Part P di Slot S bisa jalan 0 menit sampai max shift duration
            slot_capacity = int(df_slots.loc[s_idx, 'duration_min'])
            production_vars[(p_idx, s_idx)] = model.NewIntVar(0, slot_capacity, f'prod_{p_idx}_{s_idx}')

    # --- B. CONSTRAINTS (Aturan Main) ---

    # 1. Capacity Constraint: Dalam 1 Shift, total jam jalan semua part gak boleh lebih dari durasi shift
    for s_idx in slots_indices:
        slot_capacity = int(df_slots.loc[s_idx, 'duration_min'])
        model.Add(sum(production_vars[(p_idx, s_idx)] for p_idx in parts_indices) <= slot_capacity)

    # 2. Requirement Constraint: Total jam jalan Part X harus SAMA dengan kebutuhan produksinya
    for p_idx in parts_indices:
        required_minutes = int(df_requirements.loc[p_idx, 'estimated_minutes_required'])
        model.Add(sum(production_vars[(p_idx, s_idx)] for s_idx in slots_indices) == required_minutes)

    # --- C. OBJECTIVE (Strategy) ---
    # Strategy: "Earliness" -> Usahakan produksi dilakukan di Slot ID yang lebih kecil (Tanggal awal)
    # Ini biar dia gak random naruh di tanggal 30 padahal tanggal 1 kosong.
    objective_terms = []
    for p_idx in parts_indices:
        for s_idx in slots_indices:
            # Penalty score: Semakin besar s_idx (semakin akhir bulan), semakin mahal 'biaya'-nya
            objective_terms.append(production_vars[(p_idx, s_idx)] * s_idx)
            
    model.Minimize(sum(objective_terms))

    # --- D. SOLVING ---
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # --- E. EXTRACT RESULT ---
    results = []
    
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("SOLUSI DITEMUKAN! Menyusun jadwal...")
        
        for s_idx in slots_indices:
            current_date = df_slots.loc[s_idx, 'date'].strftime('%Y-%m-%d')
            shift_name = df_slots.loc[s_idx, 'shift_name']
            
            for p_idx in parts_indices:
                qty_minutes = solver.Value(production_vars[(p_idx, s_idx)])
                
                if qty_minutes > 0: # Kalau ada produksi
                    part_name = df_requirements.loc[p_idx, 'part_name']
                    part_no = df_requirements.loc[p_idx, 'part_no']
                    
                    results.append({
                        'date': current_date,
                        'shift': shift_name,
                        'part_no': part_no,
                        'part_name': part_name,
                        'minutes_allocated': qty_minutes,
                        'slot_capacity': df_slots.loc[s_idx, 'duration_min'],
                        'utilization': f"{(qty_minutes / df_slots.loc[s_idx, 'duration_min']) * 100:.1f}%"
                    })
    else:
        print("NO SOLUTION FOUND. Mungkin kapasitas mesin kurang untuk ngejar target?")

    return pd.DataFrame(results)

# ==========================================
# 5. EXECUTE
# ==========================================
# Filter dulu timeline biar cuma ambil slot mesin 250T (karena requirement kita tadi 250T semua)
# Di real case, nanti kita loop per machine_id
df_schedule = solve_production_schedule(df_timeline, df_req)

print("\n=== HASIL JADWAL FINAL ===")
# Tampilkan 15 baris pertama
print(df_schedule.head(15))

In [9]:
# ==========================================
# 6. VISUALIZATION PREP (PIVOT TABLE)
# ==========================================

def create_schedule_board(df_result, df_slots):
    # 1. Bikin Pivot: Baris = Part, Kolom = Tanggal, Isi = Shift mana aja yg jalan
    # Kita group dulu biar kalau sehari ada 3 shift, kelihatan semua
    
    # Bikin kolom 'Day' angka aja (1, 2, ..., 31) biar ringkas
    df_result['day_num'] = pd.to_datetime(df_result['date']).dt.day
    
    # Pivot Data
    # Index: Part Name & No
    # Column: Day Number
    # Value: List of Shifts (misal: "S1, S2")
    
    schedule_board = df_result.pivot_table(
        index=['part_name', 'part_no'], 
        columns='day_num', 
        values='shift',
        aggfunc=lambda x: ', '.join(x.unique()) # Gabungin text shift (ex: Shift 1, Shift 2)
    ).fillna('-') # Yang kosong kasih strip
    
    return schedule_board

# Generate Papan Jadwal
df_board = create_schedule_board(df_schedule, df_timeline)

print("\n=== SCHEDULE BOARD (Visual Excel-like) ===")
# Kita pake display() kalau di Jupyter biar rapi, atau print biasa
try:
    display(df_board)
except:
    print(df_board)


=== SCHEDULE BOARD (Visual Excel-like) ===


,day_num,1,3,4
part_name,part_no,,,
Part 116 for Avanza,7437A976XA,"Shift 1, Shift 3","Shift 2, Shift 3","Shift 1, Shift 2"
Part 63 for Pajero,MSB55992,"Shift 1, Shift 2",Shift 1,-


In [10]:
# ==========================================
# 7. VISUALISASI WARNA (THE EXCEL LOOK)
# ==========================================

def warnai_tabel(val):
    # Logic Pewarnaan:
    # 1. Kalau cell kosong ('-'), warna putih/abu
    if val == '-':
        return 'background-color: #f0f0f0; color: #d0d0d0;'
    
    # 2. Kalau ada KOMA (,), berarti dalam 1 hari/slot itu ada 2 Part yg jalan
    #    Ini representasi dari "Change Mold" atau Split Shift -> ORANGE
    elif ',' in val: 
        return 'background-color: #ffcc00; color: black; font-weight: bold;'
    
    # 3. Kalau isinya cuma 1 Shift bersih -> BIRU
    else:
        return 'background-color: #00bfff; color: white; font-weight: bold;'

print("=== JADWAL VISUAL (Sesuai Request) ===")
print("Legend: BIRU = Full Prod | ORANGE = Split/Changeover")

# Terapkan style
styled_board = df_board.style.applymap(warnai_tabel)

# Tampilkan
styled_board

=== JADWAL VISUAL (Sesuai Request) ===
Legend: BIRU = Full Prod | ORANGE = Split/Changeover


C:\Users\szcza\AppData\Local\Temp\ipykernel_17944\2139022601.py:24: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_board = df_board.style.applymap(warnai_tabel)


,day_num,1,3,4
part_name,part_no,,,
Part 116 for Avanza,7437A976XA,"Shift 1, Shift 3","Shift 2, Shift 3","Shift 1, Shift 2"
Part 63 for Pajero,MSB55992,"Shift 1, Shift 2",Shift 1,-
